# Advent of Code

## 2018-012-016
## 2018 016

https://adventofcode.com/2018/day/16

In [1]:
import re

# Load the file content
file_path = 'input.txt'
with open(file_path, 'r') as file:
    data = file.read()

# Extract samples and instructions
samples = []
instructions = []
lines = data.splitlines()
i = 0

# Parse the input for samples
while i < len(lines):
    if lines[i].startswith("Before:"):
        before = list(map(int, re.findall(r'\d+', lines[i])))
        instruction = list(map(int, re.findall(r'\d+', lines[i + 1])))
        after = list(map(int, re.findall(r'\d+', lines[i + 2])))
        samples.append((before, instruction, after))
        i += 3
    elif lines[i]:
        instructions.append(list(map(int, re.findall(r'\d+', lines[i]))))
    i += 1

# Define all opcode behaviors
def addr(registers, a, b, c):
    registers[c] = registers[a] + registers[b]

def addi(registers, a, b, c):
    registers[c] = registers[a] + b

def mulr(registers, a, b, c):
    registers[c] = registers[a] * registers[b]

def muli(registers, a, b, c):
    registers[c] = registers[a] * b

def banr(registers, a, b, c):
    registers[c] = registers[a] & registers[b]

def bani(registers, a, b, c):
    registers[c] = registers[a] & b

def borr(registers, a, b, c):
    registers[c] = registers[a] | registers[b]

def bori(registers, a, b, c):
    registers[c] = registers[a] | b

def setr(registers, a, b, c):
    registers[c] = registers[a]

def seti(registers, a, b, c):
    registers[c] = a

def gtir(registers, a, b, c):
    registers[c] = 1 if a > registers[b] else 0

def gtri(registers, a, b, c):
    registers[c] = 1 if registers[a] > b else 0

def gtrr(registers, a, b, c):
    registers[c] = 1 if registers[a] > registers[b] else 0

def eqir(registers, a, b, c):
    registers[c] = 1 if a == registers[b] else 0

def eqri(registers, a, b, c):
    registers[c] = 1 if registers[a] == b else 0

def eqrr(registers, a, b, c):
    registers[c] = 1 if registers[a] == registers[b] else 0

# Map of all opcodes
opcodes = [addr, addi, mulr, muli, banr, bani, borr, bori,
           setr, seti, gtir, gtri, gtrr, eqir, eqri, eqrr]

# Count samples behaving like 3 or more opcodes
count = 0
for before, instruction, after in samples:
    matches = 0
    for opcode in opcodes:
        registers = before[:]
        opcode(registers, *instruction[1:])
        if registers == after:
            matches += 1
    if matches >= 3:
        count += 1

count

509

In [2]:
from collections import defaultdict

def determine_opcode_mappings(samples):
    """
    Deduce the mapping of opcode numbers to functions using the samples.
    """
    possible_mappings = {i: set(opcodes) for i in range(16)}

    for before, instruction, after in samples:
        opcode_num, a, b, c = instruction
        valid_ops = set()
        for opcode in opcodes:
            registers = before[:]
            opcode(registers, a, b, c)
            if registers == after:
                valid_ops.add(opcode)
        possible_mappings[opcode_num] &= valid_ops

    # Resolve mappings to a unique function for each opcode
    resolved_mappings = {}
    while possible_mappings:
        # Find opcodes with exactly one possible function
        unique_ops = {num: list(funcs)[0] for num, funcs in possible_mappings.items() if len(funcs) == 1}
        for num, func in unique_ops.items():
            resolved_mappings[num] = func
            del possible_mappings[num]
            # Remove this function from all other sets
            for funcs in possible_mappings.values():
                funcs.discard(func)

    return resolved_mappings


def execute_program(program, opcode_mappings):
    """
    Execute the test program using the resolved opcode mappings.
    """
    registers = [0, 0, 0, 0]
    for instruction in program:
        opcode_num, a, b, c = instruction
        opcode_mappings[opcode_num](registers, a, b, c)
    return registers[0]


# Determine opcode mappings using the samples
opcode_mappings = determine_opcode_mappings(samples)

# Execute the test program
result = execute_program(instructions, opcode_mappings)
result

496